In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import Path
from tqdm.auto import tqdm
from src.config import get_dataset_path

In [ ]:

DATA_FILE = get_dataset_path("tcr_phase1_build1")  # or Path("…yourfile.hdf5")
DATA_FILE = Path("../data/2021-04-16 TCR Phase 1 Build 2.hdf5")
CLS_NAMES = {
    0: "Powder",
    1: "Printed",
    2: "Recoater Hopping",
    3: "Recoater Streaking",
    4: "Incomplete Spreading",
    5: "Swelling",
    6: "Debris",
    7: "Super-Elevation",
    8: "Spatter",
    9: "Misprint",
    10: "Over Melting",
    11: "Under Melting",
}
N_SHOW = 4               # max layers to display per class (set None to show all)
VISIBLE_DS = ["visible/0", "visible/1"]   # both RGB cameras
MAP_CM = plt.cm.viridis  # colormap for masks (optional overlay)

 - /mnt/ssd/l-pbf-dataset/2021-07-13 TCR Phase 1 Build 1.hdf5


In [5]:
def _iter_layers_with_class(h5: h5py.File, cls_id: int, chunk=128):
    """Yield (layer_idx, vis0, vis1) for every layer that contains any pixel of cls_id.

    Streams by chunks to stay memory-friendly.
    """
    ds_cls = h5[f"slices/segmentation_results/{cls_id}"]
    tot_layers = ds_cls.shape[0]
    for start in range(0, tot_layers, chunk):
        end = min(start + chunk, tot_layers)
        mask_chunk = ds_cls[start:end, ...]           # bool 3-D
        hit_layers = np.nonzero(mask_chunk.any(axis=(1,2)))[0] + start
        for idx in hit_layers:
            vis0 = h5[f"slices/camera_data/{VISIBLE_DS[0]}"][idx]
            vis1 = h5[f"slices/camera_data/{VISIBLE_DS[1]}"][idx]
            yield idx, vis0, vis1

In [6]:
with h5py.File(DATA_FILE, "r") as h5:
    n_rows = len(CLS_NAMES)
    fig, axs = plt.subplots(
        n_rows, 2, figsize=(8, n_rows * 3),
        gridspec_kw=dict(wspace=0.02, hspace=0.4),
        sharex=False, sharey=False
    )

    for row, (cls_id, cls_name) in enumerate(CLS_NAMES.items()):
        samples = list(_iter_layers_with_class(h5, cls_id))
        if not samples:
            axs[row, 0].axis("off")
            axs[row, 1].axis("off")
            continue

        # choose first N_SHOW examples
        for col, (layer_idx, vis0, vis1) in enumerate(samples[:N_SHOW]):
            # left column: visible/0
            ax0 = axs[row, 0] if col == 0 else fig.add_subplot(
                n_rows, 2*N_SHOW, row*2*N_SHOW + 2*col + 1
            )
            ax0.imshow(vis0)
            ax0.set_title(f"{cls_name} | Ch0 | L{layer_idx}")
            ax0.axis("off")

            # right column (paired): visible/1
            ax1 = axs[row, 1] if col == 0 else fig.add_subplot(
                n_rows, 2*N_SHOW, row*2*N_SHOW + 2*col + 2
            )
            ax1.imshow(vis1)
            ax1.set_title(f"{cls_name} | Ch1 | L{layer_idx}")
            ax1.axis("off")

    fig.suptitle("Visible-light examples for every defect class (both cameras)", fontsize=16)
    plt.tight_layout()
    plt.show()

TypeError: expected str, bytes or os.PathLike object, not NoneType